In [ ]:
!pip install python-dotenv
!pip install openai
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.5/387.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade openai

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
import scipy.stats
from typing import Dict, List, Union
from tqdm import tqdm
from openai import OpenAI
import os
from dotenv import load_dotenv

# Configure a API key
API_KEY = "token"
MODEL = "gpt-4"

class SICKEvaluator:
    def __init__(self, model_type: str = "gpt", threshold: float = 0.5, api_key: str = None):
        self.model_type = model_type
        self.threshold = threshold
        self.results = {}

        # Configuração do cliente OpenAI
        self.client = OpenAI(api_key=API_KEY)

        self.training_prompt = """You are a semantic similarity expert. Analyze the following pairs of sentences
        and rate their semantic similarity on a scale from 0 to 1, where:
        0: The sentences have completely different meanings
        1: The sentences have the same meaning
        Format your response as a single number between 0 and 1."""

    def prepare_prompt(self, sentence_pair: Dict) -> str:
        """
        Prepara o prompt para um par de sentenças
        """
        return f"""Sentence 1: {sentence_pair['sentence1']}
                  Sentence 2: {sentence_pair['sentence2']}
                  Rate the semantic similarity (0-1):"""

    def _process_with_gpt(self, batch: List[Dict]) -> List[float]:
        """
        Processa um lote usando GPT API
        """
        try:
            responses = []
            for pair in batch:
                prompt = self.prepare_prompt(pair)
                response = self.client.chat.completions.create(
                    model=MODEL,
                    messages=[
                        {"role": "system", "content": self.training_prompt},
                        {"role": "user", "content": prompt}
                    ],
                    temperature=0.3
                )
                responses.append(float(response.choices[0].message.content.strip()))
            return responses
        except Exception as e:
            print(f"Erro no processamento GPT: {e}")
            return [np.nan] * len(batch)

    def evaluate(self, data: List[Dict]) -> Dict:
        """
        Avalia o modelo no conjunto de dados
        """
        all_predictions = []
        ground_truth = []

        # Processa em lotes
        for batch in tqdm(data, desc="Processando pares de sentenças"):
            if self.model_type == "gpt":
                predictions = self._process_with_gpt([batch])
            else:
                predictions = self._process_with_llama([batch])

            all_predictions.extend(predictions)
            ground_truth.append(float(batch['Quality#1']))

        # Calcula métricas
        self.results = self.calculate_metrics(all_predictions, ground_truth)
        return self.results

    def calculate_metrics(self, predictions: List[float], ground_truth: List[float]) -> Dict:
        """
        Calcula todas as métricas relevantes
        """
        # Remove pares com NaN antes do cálculo
        valid_pairs = [(p, t) for p, t in zip(predictions, ground_truth) if not np.isnan(p)]
        if not valid_pairs:
            return {
                'pearson': np.nan,
                'spearman': np.nan,
                'accuracy': np.nan,
                'precision': np.nan,
                'confusion_matrix': None
            }

        pred_clean, true_clean = zip(*valid_pairs)

        # Métricas de correlação
        pearson = scipy.stats.pearsonr(pred_clean, true_clean)[0]
        spearman = scipy.stats.spearmanr(pred_clean, true_clean)[0]

        # Converte para classificação binária usando threshold
        pred_binary = [1 if p >= self.threshold else 0 for p in pred_clean]
        true_binary = [1 if t >= self.threshold else 0 for t in true_clean]

        # Métricas de classificação
        accuracy = accuracy_score(true_binary, pred_binary)
        precision = precision_score(true_binary, pred_binary, average='weighted')
        conf_matrix = confusion_matrix(true_binary, pred_binary)

        return {
            'pearson': pearson,
            'spearman': spearman,
            'accuracy': accuracy,
            'precision': precision,
            'confusion_matrix': conf_matrix
        }

    def print_results(self):
        """
        Imprime os resultados de forma organizada
        """
        print("\n=== Resultados da Avaliação ===")
        print(f"Correlação de Pearson: {self.results['pearson']:.4f}")
        print(f"Correlação de Spearman: {self.results['spearman']:.4f}")
        print(f"Acurácia: {self.results['accuracy']:.4f}")
        print(f"Precisão: {self.results['precision']:.4f}")
        print("\nMatriz de Confusão:")
        print(self.results['confusion_matrix'])

def main():
    # Configuração de arquivos e tamanho da amostra
    NOME_ARQUIVO_TREINAMENTO = "sick_train.csv"
    NOME_ARQUIVO_TESTE = "sick_test.csv"
    TAMANHO_AMOSTRA = 100  # Define o tamanho máximo da amostra

    # Carrega os dados
    try:
        df = pd.read_csv(NOME_ARQUIVO_TESTE)
        print(f"Arquivo de teste carregado, total de {len(df)} pares")
    except FileNotFoundError:
        print(f"Arquivo {NOME_ARQUIVO_TESTE} não encontrado, tentando carregar arquivo de treino")
        try:
            df = pd.read_csv(NOME_ARQUIVO_TREINAMENTO)
            print(f"Arquivo de treino carregado, total de {len(df)} pares")
        except FileNotFoundError:
            raise FileNotFoundError("Nenhum arquivo de dados encontrado")

    # Renomeia as colunas para facilitar o acesso
    df.columns = ['Quality', '#1 ID', '#2 ID', '#1 String', '#2 String']

    print("\nDistribuição das classes de similaridade:")
    print(df['Quality'].value_counts())

    # Amostragem aleatória estratificada
    df_sample = df.groupby('Quality').apply(
        lambda x: x.sample(min(len(x), max(1, TAMANHO_AMOSTRA // len(df['Quality'].unique()))))
    ).reset_index(drop=True)

    print(f"\nTamanho da amostra selecionada: {len(df_sample)} pares")
    print("Distribuição das classes na amostra:")
    print(df_sample['Quality'].value_counts())

    # Prepara os dados da amostra
    data = []
    for _, row in df_sample.iterrows():
        pair = {
            'sentence1': row['#1 String'],
            'sentence2': row['#2 String'],
            'Quality#1': row['Quality'],
            'id1': row['#1 ID'],
            'id2': row['#2 ID']
        }
        data.append(pair)

    # Inicializa e executa o avaliador
    evaluator = SICKEvaluator(model_type="gpt", threshold=0.5)

    # Testa a conexão com a API
    try:
        test_response = evaluator.client.chat.completions.create(
            model=MODEL,
            messages=[{"role": "user", "content": "Test"}]
        )
        print("\nConexão com API bem sucedida")
    except Exception as e:
        print(f"\nErro na conexão com API: {e}")
        return

    # Executa a avaliação
    print("\nIniciando avaliação dos pares de sentenças...")
    results = evaluator.evaluate(data)
    evaluator.print_results()

if __name__ == "__main__":
    main()


Arquivo de teste carregado, total de 1332 pares

Distribuição das classes de similaridade:
Quality
1    704
0    628
Name: count, dtype: int64

Tamanho da amostra selecionada: 100 pares
Distribuição das classes na amostra:
Quality
0    50
1    50
Name: count, dtype: int64


<ipython-input-4-f96228e6e584>:156: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sample = df.groupby('Quality').apply(



Conexão com API bem sucedida

Iniciando avaliação dos pares de sentenças...


Processando pares de sentenças: 100%|██████████| 100/100 [01:02<00:00,  1.60it/s]


=== Resultados da Avaliação ===
Correlação de Pearson: 0.0869
Correlação de Spearman: 0.1125
Acurácia: 0.5000
Precisão: 0.5000

Matriz de Confusão:
[[14 36]
 [14 36]]


In [ ]:
!pip3 install llamaapi

In [ ]:
from llamaapi import LlamaAPI
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
import scipy.stats
from typing import Dict, List, Union
from tqdm import tqdm

LLAMA_API_TOKEN = "token"

class LlamaSimilarityEvaluator:
    def __init__(self, threshold: float = 0.5):
        self.llama = LlamaAPI(LLAMA_API_TOKEN)
        self.threshold = threshold
        self.results = {}

    def prepare_prompt(self, sentence_pair: Dict) -> Dict:
        """
        Prepara o prompt para o formato da Llama API
        """
        return {
            "model": "llama3.1-70b",  # Usando o modelo mais recente e capaz
            "messages": [
                {"role": "system", "content": "You are a semantic similarity expert. Rate the similarity between sentences on a scale from 0 to 1."},
                {"role": "user", "content": f"""Rate the semantic similarity between these sentences:
                Sentence 1: {sentence_pair['sentence1']}
                Sentence 2: {sentence_pair['sentence2']}

                Return only a number between 0 and 1, where:
                0: Completely different meanings
                1: Identical meanings"""}
            ],
            "stream": False,
            "temperature": 0.1  # Baixa temperatura para respostas mais consistentes
        }

    def process_pair(self, pair: Dict) -> float:
        """
        Processa um par de sentenças usando a Llama API
        """
        try:
            api_request = self.prepare_prompt(pair)
            response = self.llama.run(api_request)

            # Extrai o valor numérico da resposta
            similarity = float(response.json()['choices'][0]['message']['content'].strip())
            return similarity
        except Exception as e:
            print(f"Erro no processamento: {e}")
            return np.nan

    def evaluate(self, data: List[Dict]) -> Dict:
        """
        Avalia o conjunto de dados
        """
        predictions = []
        ground_truth = []

        print("Processando pares de sentenças...")
        for pair in tqdm(data):
            similarity = self.process_pair(pair)
            if not np.isnan(similarity):
                predictions.append(similarity)
                # Normaliza ground truth de [1-4] para [0-1]
                gt = (float(pair['Quality#1']) - 1) / 3
                ground_truth.append(gt)

        self.results = self.calculate_metrics(predictions, ground_truth)
        return self.results

    def calculate_metrics(self, predictions: List[float], ground_truth: List[float]) -> Dict:
        """
        Calcula métricas de avaliação
        """
        if not predictions or not ground_truth:
            return {
                'pearson': np.nan,
                'spearman': np.nan,
                'accuracy': np.nan,
                'precision': np.nan,
                'confusion_matrix': None
            }

        # Métricas de correlação
        pearson = scipy.stats.pearsonr(predictions, ground_truth)[0]
        spearman = scipy.stats.spearmanr(predictions, ground_truth)[0]

        # Classificação binária
        pred_binary = [1 if p >= self.threshold else 0 for p in predictions]
        true_binary = [1 if t >= self.threshold else 0 for t in ground_truth]

        # Métricas de classificação
        accuracy = accuracy_score(true_binary, pred_binary)
        precision = precision_score(true_binary, pred_binary, average='weighted', zero_division=0)
        conf_matrix = confusion_matrix(true_binary, pred_binary)

        return {
            'pearson': pearson,
            'spearman': spearman,
            'accuracy': accuracy,
            'precision': precision,
            'confusion_matrix': conf_matrix
        }

    def print_results(self):
        """
        Imprime resultados formatados
        """
        print("\n=== Resultados da Avaliação ===")
        print(f"Correlação de Pearson: {self.results['pearson']:.4f}")
        print(f"Correlação de Spearman: {self.results['spearman']:.4f}")
        print(f"Acurácia: {self.results['accuracy']:.4f}")
        print(f"Precisão: {self.results['precision']:.4f}")
        print("\nMatriz de Confusão:")
        print(self.results['confusion_matrix'])

def load_dataset(filename: str, sample_size: int = None) -> List[Dict]:
    """
    Carrega e prepara o dataset
    """
    df = pd.read_csv(filename)
    df.columns = ['Quality', '#1 ID', '#2 ID', '#1 String', '#2 String']

    if sample_size:
        df_sample = df.groupby('Quality', group_keys=False).apply(
            lambda x: x.sample(min(len(x), max(1, sample_size // len(df['Quality'].unique()))))
        ).reset_index(drop=True)
    else:
        df_sample = df

    print(f"\nDistribuição das classes na amostra:")
    print(df_sample['Quality'].value_counts().sort_index())

    data = []
    for _, row in df_sample.iterrows():
        pair = {
            'sentence1': row['#1 String'],
            'sentence2': row['#2 String'],
            'Quality#1': row['Quality'],
            'id1': row['#1 ID'],
            'id2': row['#2 ID']
        }
        data.append(pair)

    return data

def main():
    print("Iniciando avaliação de similaridade com Llama API...")

    # Carrega dados
    try:
        data = load_dataset('cohquad_sick_train.csv', sample_size=100)
        print(f"Total de pares carregados: {len(data)}")
    except FileNotFoundError:
        print("Arquivo de dados não encontrado!")
        return

    # Inicializa e executa avaliador
    evaluator = LlamaSimilarityEvaluator(threshold=0.5)

    # Testa conexão com API
    try:
        test_request = {
            "model": "llama3.1-70b",
            "messages": [{"role": "user", "content": "Test"}],
            "stream": False
        }
        evaluator.llama.run(test_request)
        print("\nConexão com Llama API bem sucedida")
    except Exception as e:
        print(f"\nErro na conexão com Llama API: {e}")
        return

    # Executa avaliação
    print("\nIniciando avaliação dos pares de sentenças...")
    evaluator.evaluate(data)
    evaluator.print_results()

if __name__ == "__main__":
    main()

Iniciando avaliação de similaridade com Llama API...

Distribuição das classes na amostra:
Quality
0    50
1    50
Name: count, dtype: int64
Total de pares carregados: 100


<ipython-input-3-bba650c92ffb>:125: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sample = df.groupby('Quality', group_keys=False).apply(



Conexão com Llama API bem sucedida

Iniciando avaliação dos pares de sentenças...
Processando pares de sentenças...


100%|██████████| 100/100 [03:28<00:00,  2.09s/it]


=== Resultados da Avaliação ===
Correlação de Pearson: 0.1525
Correlação de Spearman: 0.1766
Acurácia: 0.4400
Precisão: 1.0000

Matriz de Confusão:
[[44 56]
 [ 0  0]]
